# **Project Name**    - Rosamann Retails Sales Prediction



##### **Project Type**    - Regression
##### **Contribution**    - Individual
##### **Member 1 -**  - Jayesh Prakash Dahiwale


# **Project Summary -**

**Rossmann operates over 3,000 drug stores in 7 european countries. So our porject aim is to predict their daily sales for upto six weeks in advance. Stores sales are influenced by many factors, including promotions, competitions,school and state holidays, seasonality and locality. So In this project we have been provided the data from 1,500 Rossmann stores. So our task is to forecast the "Sales" column for the test** 
##<p>Provided there are two datasets, one is **Store** dataset having <b>1,115</b> observations in it with <b>10</b> columns and It gives us static information about each store such as the model and assortment of the store, information about the nearest competitor store, and whether or not they participate in the consecutive promotion "Promo2". Largely we're looking at numerical and date data, but Store Type and Assortment are flagged with letters to indicate store models and assorment level, per the variable explanations, as well as the PromoInterval column listing abbreviated months.</p>
## <p>Other datatset is about **Sales** dataset having <b>1,017,209</b> observations in it with <b>9</b> columns and It gives us static information about each store such as the model and assortment of the store, information about the nearest competitor store, and whether or not they participate in the consecutive promotion "Promo2". Largely we're looking at numerical and date data, but Store Type and Assortment are flagged with letters to indicate store models and assorment level, per the variable explanations, as well as the PromoInterval column listing abbreviated months.</p>

# **GitHub Link -**

https://github.com/Jayeshdahiwale/RegressionCapstoneProjectII

# **DataSet Link -**


Dataset DriveLink:https://drive.google.com/drive/folders/1XtTzgmtM-e9jGHbIq8pLzIbo3C8NF1T2?usp=share_link



## **Questions for analysis are as follows**
**1) Visualising the distribution of "Sales" & "Customers"?**<br>
**2) Statistics of Sales column ?** <br>
**3) Which rows are unnecessary and need to be removed ??** <br>
**4) What are the outliers ?** <br>
**5) Establishing relationship between Sales and Customers ?** <br>
**6) How stores are performing in Sales by month based on Assortment type ?**<br>
**7) How UPT metric compares across stores of different assortment types ?**<br>
**8)Correlation between competition distance and UPT metric?**<br>
**9)Which linear regression model is best?**<br>




#### **Define Your Business Objective?**

Increase the number of sales by predicting the rates at at optimal rate and finding the best suitalbe condition which attract the customers thereby increasing the profit for the Drug Store.

## **Importing the required packages**

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import math

We'll begin with the required libraries and reading our .csv files

In [ ]:
# mount the google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Reading the dataset file
# dataset file googgle drive link : https://drive.google.com/file/d/185htr6OyxAZ0xb9QwxHpsUn8eTlq58v5/view?usp=share_link
working_directory_path = '/content/drive/MyDrive/MachineLearningAlmabetterJourney/CapstoneProject2/'



In [ ]:
store_lookup = pd.read_csv(working_directory_path + 'store.csv')

In [ ]:
sales_data = pd.read_csv(working_directory_path + 'Rossmann Stores Data.csv')


In [ ]:
store_lookup.head()

In [ ]:
store_lookup.shape

In [ ]:
store_lookup.info()

In [ ]:
def null_cols(data):
  columns = data.columns
  null_cols = [(col,data[col].isna().sum()) for col in columns if data[col].isna().sum()!=0]
  return null_cols

In [ ]:
null_cols(store_lookup)

We can see in store lookup table there are total 6 null columns

In [ ]:
# Lets see the sales data
sales_data.head()

In [ ]:
sales_data.shape

In [ ]:
sales_data.info()

These is no null columns in this dataset

In [ ]:
#lets see the starting date and end date of sales data
print(sales_data['Date'].min())
print(sales_data['Date'].max())

**Now we will merge the two data resources so we can more easily work with them. We will join the tables baes on the shared store column, which is a foreign key in the sales_data table and primary key in the stores_llokup table, so we'll validate the merge baed on this many to one relationship**

In [ ]:
merged_sales = sales_data.merge(store_lookup,how='left',on='Store',validate = 'many_to_one')

In [ ]:
merged_sales.info()

In [ ]:
merged_sales.shape

In [ ]:
merged_sales.head()

In [ ]:
merged_sales.describe()

In [ ]:
merged_sales.info()

**Lets check the skewness of columns i.e.** <br>
 "Day of Week","Sales","Customers","Competion Distance"

In [ ]:
merged_sales[['DayOfWeek','Sales','Customers','CompetitionDistance']].skew()

we can see here competition distance,customers are positively skewed<br>


# **<p>Lets check the distribution bell peak using kurotsis</p>**

In [ ]:
merged_sales[['DayOfWeek','Sales','Customers','CompetitionDistance']].kurtosis()

We can see the that competition distance,and customers has highest peak...And sales and Day of week has negative kurtosis

# **Question 1: Visualising the distribution of Sales and Customer columns**

# **<p>Lets visualize the skewness and kurtosis</p>**

In [ ]:
fig,axes = plt.subplots(1,2)
sns.distplot(merged_sales['Sales'],color="y",ax=axes[0])
sns.distplot(merged_sales['Customers'],color="y",ax=axes[1])
fig.set_size_inches(15,5)

Histograms of our Sales and Customers values show us a positive skew and high kurtosis.

# **Question 2: Statistics of Sales column**

# <p>**Next we will take a closer look at our statistics for sales column.**</p>

In [ ]:
print("skew     " + str(round(merged_sales['Sales'].skew(),6)))
print("kurtosis " + str(round(merged_sales['Sales'].kurtosis(),6)))
print(merged_sales['Sales'].describe().round(3))
print("mode     " + str(merged_sales['Sales'].mode()))

We can see our sales figures have a slightly positive skew, with the **mean (5773.819)** only slightly larger than the **median (5744.000)**, suggesting most outliers are to the right of the mean.

High kurtosis indicates it's **leptokurtic** with the likelihood of heavy tails and outliers that may be extreme. Considering our min and max values of **0** and **41,551** sales, we aren't surprised to see there may be some extreme outliers.

The max value well above the mean of 5,773.819 and outside the **standard deviation of 3849.926** helps us see how our mean ends up getting pulled slightly to the right for our positive skew.

There is no mode as we don't have any stores recording the exact same number of sales on any days, which isn't surprising.

# Data Cleaning

#<p>Datetime</p>

First we'll convert our Date column to datetime for easier filtering later on.

With an .info() call we can confirm our Date column was successfully converted to datetime.

We'll also expand our Date column into separate Month, Day of Month, and Year columns for easier filtering.

In [ ]:
merged_sales['Date'] = pd.to_datetime(merged_sales['Date'], format="%Y-%m-%d", errors='raise')

In [ ]:
merged_sales.info()

In [ ]:
merged_sales["Year"] = merged_sales["Date"].dt.year
merged_sales["Month"] = merged_sales["Date"].dt.month
merged_sales["DayOfMonth"] = merged_sales["Date"].dt.day

# **Duplicates**

We'll do a quick check for and delete any rows that are complete duplicates of another row, as we should only have one entry for each store and date.

In [ ]:
len(merged_sales[merged_sales.duplicated()])

# **Text and Expected Value**

In [ ]:
merged_sales.loc[merged_sales['StateHoliday']==0,'StateHoliday'] = '0'

Next we'll check our four columns that contain strings to make sure we have no inappropriately entered data. Using str.strip() to remove any accidental leading or trailing spaces.

In [ ]:
for col in merged_sales:
  
  if merged_sales[col].dtype == object and col != 'StateHoliday':
    merged_sales[col] = merged_sales[col].str.strip()

Next we'll look at value_counts for those columns and confirm only expected values are found.

In [ ]:
for col in merged_sales:
  if merged_sales[col].dtype==object:
    print(merged_sales[col].value_counts().sort_index())

In [ ]:
merged_sales.info()

Everything is looking as expected in our columns that contain text, and because each column only has 3-4 unique values we can see we don't need to worry about changing anything to lower, upper, or proper case.

We'll also do a quick check of columns we expect to only have a few unique values or binary flags, such as DayOfWeek or Promo, to make sure there's nothing unexpected there.

In [ ]:
check_cols = [ 'Open', 'Promo', 'Promo2', 'SchoolHoliday',  'DayOfWeek', 'CompetitionOpenSinceMonth',  'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear']
for col in check_cols:
    print(col)
    print(sorted(merged_sales[col].unique()))

We see the relevant binary flags, day of weeks, week, month, and year numeric values we expect.

Because the CompetitionOpenSinceMonth, CompetitionOpenSinceYear, Promo2SinceWeek, and Promo2SinceYear columns are only using whole numbers and they are a discrete value, we will change them from floats to integers.

In [ ]:
merged_sales['CompetitionOpenSinceMonth'] = merged_sales['CompetitionOpenSinceMonth'].convert_dtypes()
merged_sales['CompetitionOpenSinceYear'] = merged_sales['CompetitionOpenSinceYear'].convert_dtypes()
merged_sales['Promo2SinceWeek'] = merged_sales['Promo2SinceWeek'].convert_dtypes()
merged_sales['Promo2SinceYear'] = merged_sales['Promo2SinceYear'].convert_dtypes()

In [ ]:
merged_sales.info()

Now we can see that CompetitionDistance, our only continuous value, is our only column with floats.

#  **Question3):Removing rows and columns**

Below I look at entries for stores on days they were closed.

For the purposes of our analysis I've chosen to drop these rows, as no sales are recorded on days stores are closed. The zero sales recorded for each of these rows lowers the average sales, and we can see this by comparing the mean Sales for all entries in our table to the mean Sales of only days that stores were open. If we filter for entries of stores that are closed we'll see a return of **172,817** rows, all of which record the expected 0 sales, lowering our mean Sales statistic.

**The potential information lost here is if we want to compare stores based on the number of days they are open or closed**, but that is beyond the scope of our analysis for now. To avoid losing this information we will make a copy of our dataframe with only the days stores are open, to further be referred to as sales, rather than altering merged_sales in case we wish to access this data at a later time.

In [ ]:
## mean sales including entries for days stores are closed
merged_sales['Sales'].mean()

In [ ]:
#Not lets check the mean only for the data when the store is open
merged_sales[merged_sales['Open']==1].Sales.mean()

In [ ]:
## confirming all entries where the store is  marked as closed have 0 sales
merged_sales.loc[merged_sales["Open"] == 0, ['Sales', 'Customers']].value_counts()


In [ ]:
## creating new sales dataframe with only entries for days stores are open
sales = merged_sales.drop(index=(merged_sales[merged_sales["Open"] == 0]).index, axis=1)

Now as we chose to delete any rows for days a store wasn't open, our Open column should only contain the value 1 now and is redundant, so we'll remove that

In [ ]:
sales.drop(columns=["Open"], inplace=True)

# **Question 4 : What are the outliers in the dataset ?**


Next we'll take a look at any outliers we may need to treat.

In [ ]:
sales.plot(y=['Sales', 'Customers', 'CompetitionDistance'], 
           kind='box', subplots=True, layout=(2,2), figsize=(15,15))

From the box plots above we can see that Sales, Customers, and CompetitionDistance all appear to have significant outliers, so we'll explore further by calculating and investigating the outliers for each one.


# **Sales Outliers**



We will start with the Sales column.

In [ ]:
def calculate_outlier(df,column): ## function for calculating outliers
    Q3 = df[column].quantile(0.75)
    Q1 = df[column].quantile(0.25)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    percent_outliers = round(((df[df[column] > upper].shape[0]) + (df[df[column] < lower].shape[0])) / df.shape[0] * 100, 2)
    return lower, upper, percent_outliers

In [ ]:
col = 'Sales'
lower_sales, upper_sales, percent_outliers_sales = calculate_outlier(sales, col)

print("lower band = " + str(lower_sales))
print("upper band = " + str(upper_sales))
print("percentage of sales that are outliers = " + str(percent_outliers_sales) + "%")
print('Number of observations in which sales leass than <0 are '+str(len(sales[sales.Sales <0])))

We know from our summary statistics that there aren't any sales below 0, so we'll just look at the upper outliers that we've calculated for the Sales column.

In [ ]:
sales[sales[col] > upper_sales]

While 30,769 is a lot of values, we can see from our calculte_outlier function that these outliers only account for 3.64% of all our sales values.

We'll look further to see if we see any trends with the outliers based on Month or Type of Store.

In [ ]:
sales_outliers_by_month = pd.pivot_table((sales.loc[sales[col] > upper_sales]), index='Month', values='Sales', aggfunc='count')

sales_outliers_by_month.plot(y='Sales', kind='bar', figsize=(10,5), title="# of Sales Outlier Entries by Month")
plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1.1))
plt.show()

In [ ]:
sales_outliers_by_stype = pd.pivot_table((sales.loc[sales[col] > upper_sales]), index='StoreType', values='Sales', aggfunc='count')

sales_outliers_by_stype.plot(y='Sales', kind='bar', figsize=(6,6), 
                             title="# of Sales Outlier Entries by Store Type", 
                             color=['blue','red','green','orange'])
plt.ylabel('Sales')
plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1.1))
plt.show()

When we look at the Sales outliers by month, we see the most represented month is **December at 17.33%**, which is unsurprising given the Christmas holidays. However, when we look at the outliers by Store Type we see that the **61.71% majority** are coming from **Type A stores**, while **Type B, C, D** are more equally represented at **11-15%**. This suggests that Type A stores may be the best performers in regards to outstanding sales days, and is worth looking into further.

Below we will treat our Sales outliers by imputing them with our upper range value we calculated earlier, **13611.5**, rounded up to **13612** as our Sales column is a measure of discrete values using whole numbers. As these outliers represent exceptionally high sales day, they are intended to be high numbers, but we would like to treat the outliers to limit their influence on any future modelling. As such imputing with our upper range value feels more appropriate than using our mean Sales value.

We also save this a new dataframe going forward, to further be referenced to as sales_treated, so that we can preserve our sales dataframe with the outliers intact, should we wish to investigate them further.

In [ ]:
sales_treated = sales.copy()

In [ ]:
sales_treated.loc[sales_treated[col] > upper_sales, 'Sales'] = 13612

Further check whether our imputation worked or not

In [ ]:
sales_treated[sales_treated['Sales'] > 13612]

# **Customer Outliers**

Next we'll calculate and investigate our Customer outliers.

In [ ]:
col = 'Customers'
lower_cust, upper_cust, percent_outliers_cust = calculate_outlier(sales_treated, col)

print(str(lower_cust) + ", " + str(upper_cust) +", " + str(percent_outliers_cust) + "%")
print('Number of rows when customer < 0 : ' + str(len(sales[sales_treated['Customers']<0])))

Similar to Sales, we know from our summary statistics that we don't have any Customer values below 0, so we'll just look at our upper range value.

In [ ]:
sales_treated[sales_treated['Customers'] > upper_cust]

We Can see right away that several of these entries have a **Sales value of 13,612**, which we know to be our newly imputed upper range value for Sales outliers. We expect a high correlation between Customers driving Sales, so we'll check to see how much crossover we have between our Sales and Customers outliers.\

In [ ]:
sales_treated[(sales_treated['Customers'] > upper_cust) & (sales_treated['Sales'] == 13612)]

We can see a crossover of **21,420 rows**, or approximately **52%** of our Customer outlier entries are also Sales outlier entries.

We will also investigate how these Customer outliers break down by Month and StoreType just as we did with our Sales outliers.

In [ ]:
cust_outliers_by_month = pd.pivot_table((sales_treated.loc[sales_treated[col] > upper_cust]), index='Month', values='Customers', aggfunc='count')

cust_outliers_by_month.plot(y='Customers', kind='bar', figsize=(10,5), title="# of Customer Outlier Entries by Month")
plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1.1))
plt.show()

In [ ]:
cust_outliers_by_stype = pd.pivot_table((sales_treated.loc[sales_treated[col] > upper_cust]), index='StoreType', values='Customers', aggfunc='count')

cust_outliers_by_stype.plot(y='Customers', kind='bar', figsize=(6,6), 
                             title="# of Customer Outlier Entries by Store Type", 
                             color=['blue','red','green','orange'])
plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1.1))
plt.show()

**December** is our most represented month for Customer outliers, similar to our Sales outliers, but it's percentage of outliers is less than we saw with our Sales.

We also see **store Type A** with the strongest showing when we break down the outliers by store type. Much like the Sales outliers **Type A stores** represent a **strong 60%+** of the outliers. Surprisingly, **Type D stores** represent a tiny **1.02%** of these Customer outliers, where as they represented the second largest percentage of Sales outliers at **15.29%**. Further investigation into the number of items bought (Sales) per transaction (Customer) may prove insightful.

Similar to our Sales outliers, we will also limit our Customer outliers to our calculated upper range, by imputing them to **1,454,** so as to limit their influence but also indicate that they're meant to be high numbers.

In [ ]:
sales_treated.loc[sales_treated['Customers'] > upper_cust, 'Customers'] = 1454

In [ ]:
sales_treated.loc[sales_treated['Customers'] > upper_cust, 'Customers']

# **Missing Values**

Next we'll take a look at missing values. We'll start by assessing how many we have and where.

In [ ]:
sales_treated.isna().sum()

In [ ]:
(sales_treated.isna().sum() * 100 / sales_treated.shape[0]).round(2)   ## missing values as a % of all values in the column

While we see a rather high 50% missing values rate for Promo2SinceWeek and Promo2SinceYear we can check and see if these all just correspond to stores that aren't running Promo2, and hence would not be expected to have valid data for these columns.

In [ ]:
print(sales_treated[sales_treated['Promo2'] == 0].shape[0])
print(sales_treated['Promo2SinceWeek'].isna().sum())
print(sales_treated['Promo2SinceYear'].isna().sum())

As the number of entries with missing **Promo2SinceWeek** and **Promo2SinceYear** values corresponds with the number of entries where the store isn't participating in Promo2, we can rest assured that the lack of data here is appropriate.



The other area we see missing data is in relation to competition stores, mostly in the two columns relating when the nearest competitor store opened, and to much lesser extent in **CompetitionDistance**

The CompetitionDistance values may be used for analyzing how stores perform based on how close their competition is, and as such missing values could skew such analysis. 0 would be an inappropriate replacement for these null values as it would indicate that the competition stores are incredibly close. As such we'll look to replace these missing values with the mean CompetitionDistance based on the StoreType.

In [ ]:
mean_type_a = round(store_lookup.loc[(store_lookup['StoreType'] == 'a'), 'CompetitionDistance'].mean(), 1)
mean_type_b = round(store_lookup.loc[(store_lookup['StoreType'] == 'b'), 'CompetitionDistance'].mean(), 1)
mean_type_c = round(store_lookup.loc[(store_lookup['StoreType'] == 'c'), 'CompetitionDistance'].mean(), 1)
mean_type_d = round(store_lookup.loc[(store_lookup['StoreType'] == 'd'), 'CompetitionDistance'].mean(), 1)

print("The mean Compeition Distance for stores of type A is " + str(mean_type_a))
print("The mean Compeition Distance for stores of type B is " + str(mean_type_b))
print("The mean Compeition Distance for stores of type C is " + str(mean_type_c))
print("The mean Compeition Distance for stores of type D is " + str(mean_type_d))

In [ ]:
sales_treated.loc[sales_treated['StoreType'] == 'a'] = sales_treated.loc[sales_treated['StoreType'] == 'a'].fillna(value={"CompetitionDistance" : mean_type_a})
sales_treated.loc[sales_treated['StoreType'] == 'b'] = sales_treated.loc[sales_treated['StoreType'] == 'b'].fillna(value={"CompetitionDistance" : mean_type_b}) 
sales_treated.loc[sales_treated['StoreType'] == 'c'] = sales_treated.loc[sales_treated['StoreType'] == 'c'].fillna(value={"CompetitionDistance" : mean_type_c}) 
sales_treated.loc[sales_treated['StoreType'] == 'd'] = sales_treated.loc[sales_treated['StoreType'] == 'd'].fillna(value={"CompetitionDistance" : mean_type_d}) 

In [ ]:
sales_treated.isna().sum()

Next we will replace the null values in our PromoInterval column. As this column contains strings listing the months the Promo2 starts anew, we will replace the missing values with a string 'NA' for Not Applicable.

In [ ]:
sales_treated.loc[sales_treated['PromoInterval'].isna(), 'PromoInterval'] = "NA"

In [ ]:
sales_treated['PromoInterval'].value_counts()

For the missing values in **'CompetitionOpenSinceYear'** & **'Promo2SinceYear'** it would be inappropriate to replace them with zeros, as further calculations with the years in that column could lead to values suggesting a competition store has been open **2022** years. As such, we'll impute these missing values with the current year, so if calculations are done to find how long the nearest competition store has been open since these instances of no competition store nearby will return zero years.

In the same manner we will impute the missing values in **'CompetitionOpenSinceMonth'** with the **current month**, and 'Promo2SinceWeek' with the current week.

In [ ]:
sales_treated.loc[sales_treated['CompetitionOpenSinceYear'].isna(), 'CompetitionOpenSinceYear'] = dt.datetime.now().year

sales_treated.loc[sales_treated['Promo2SinceYear'].isna(), 'Promo2SinceYear'] = dt.datetime.now().year

sales_treated.loc[sales_treated['CompetitionOpenSinceMonth'].isna(), 'CompetitionOpenSinceMonth'] = dt.datetime.now().month

sales_treated.loc[sales_treated['Promo2SinceWeek'].isna(), 'Promo2SinceWeek'] = dt.datetime.now().isocalendar()[1]

In [ ]:
# lets see the null counts
sales_treated.isna().sum()

# **Question 5: Establising relationship between Sales and Customers ?**

 **Adding an UPT column**

Based on our investigation of Sales and Costumers outliers, particularly how they're represented by different store types, I thought it would be worthwhile to add a calculated column to give us some insight into the relationship between Customers and Sales for each store and day.

In [ ]:
(sales_treated['Customers'] >= sales_treated['Sales']).value_counts()

We can see that for almost all entries we have in our dataframe, the number of Sales at a given store is greater than the number of Customers. Suggesting that the Customers value is derived by how many transactions there are at a store, and the Sales value is indicative of how many individual items are sold. Thus we can calculate the average number of items sold for each transaction as Units Per Transaction (UPT).

First we'll quickly investigate the 54 rows where there aren't more Sales than Customers.

In [ ]:
sales_treated[sales_treated['Customers'] >= sales_treated['Sales']]

We can see the majority of these days are entries with both zero Sales and zero Customers recorded. This seems odd for a day that the store is open. A quick check of merged_sales, which still has the Open column, gives us the same results and assures us that the stores are indeed marked as open on these days.

In [ ]:
merged_sales[(merged_sales['Customers'] >= merged_sales['Sales']) & merged_sales['Open'] == 1]

I can only speculate as to why we have entries for stores that are open but aren't recording any sales, perhaps a stocktake day? Let's also look at the cases that aren't zero Sales and zero Customers.

In [ ]:
sales_treated[sales_treated['Customers'] > sales_treated['Sales']]

I'm unsure of why we have two days with zero Sales and a small handful of Customers. As I have no explanation for these 2 days, nor the 52 other open days with zero Sales and Customers, I don't feel comfortable deleting them.

This does pose a small problem for calculating our average UPT, however. As such we will create our UPT column by dividing the day's Sales by the days Customers to find the average Units Per Transaction for each day and store. The resulting 52 null values will be imputed with a zero to reflect the zero Sales for those entries.

In [ ]:
sales_treated['UPT'] = sales_treated['Sales'] / sales_treated['Customers']

In [ ]:
sales_treated['UPT'].isna().sum()

In [ ]:
sales_treated.loc[sales_treated['UPT'].isna(), 'UPT'] = 0

In [ ]:
sales_treated['UPT'].isna().sum()

We'll also add this UPT column to our sales table which still contains our Sales and Customers outliers, so we can explore the UPT with the outliers as well.

In [ ]:
sales['UPT'] = sales['Sales'] / sales['Customers']
sales.loc[sales['UPT'].isna(), 'UPT'] = 0

# **Review of Summary Statistics Post Cleaning**

Let's quickly review some of our summary statistics now that we've cleaned our data a bit. We can look at the summary statistics for sales to see them as they were before treating outliers and imputing for missing data, and compare them to the sales_treated summary statistics to see how they've changed.

In [ ]:
merged_sales[['Sales', 'Customers', 'CompetitionDistance']].describe() ## BEFORE cleaning


In [ ]:
sales_treated[['Sales', 'Customers', 'CompetitionDistance', 'UPT']].describe() ## AFTER cleaning

We can see that the cleaning has narrowed our standard deviation for Sales and Customers, as well as raising the mean after removing the entries for closed stores.

In [ ]:
merged_sales[['Sales', 'Customers', 'CompetitionDistance']].skew() ## BEFORE cleaning

In [ ]:
sales_treated[['Sales', 'Customers', 'CompetitionDistance', 'UPT']].skew() ## AFTER cleaning

In [ ]:
merged_sales[['Sales', 'Customers', 'CompetitionDistance']].kurtosis() ## BEFORE cleaning

In [ ]:
sales_treated[['Sales', 'Customers', 'CompetitionDistance', 'UPT']].kurtosis() ## AFTER cleaning

In [ ]:
fig,axes = plt.subplots(1,2)
sns.distplot(sales_treated['Sales'],color="y",ax=axes[0])
sns.distplot(sales_treated['Customers'],color="y",ax=axes[1])
fig.set_size_inches(15,5)

Histograms of our Sales and Customers values shows us a slight positive skew, which changed a little after our cleaning, and we see a more significant change in kurtosis being lowered. We also see the effect of imputing our outliers with our upper range limit on the right side of either histogram.

 we now got results that are looking far closer to a standard normalization.

## **Exploratory Insights**

## **Question 6: How stores perform in Sales by month based of Assortment type?**

Let's explore how stores perform in Sales by month, based on Assortment type. We know that Assortment type A offers a "basic" assortment of merchandise, Type B offers and "extra" assortment, and type C offers an "extended" assortment.

Because our data ranges from Jan. 1, 2013 - July 31, 2015, we will exclude the 2015 data for now so as we are only looking at a complete years' worth of numbers.

In [ ]:
assortment_pivot_total_sales = pd.pivot_table((sales_treated[sales_treated['Year'] < 2015]), ## Excluding 2015 data
               index='Month', values='Sales', columns='Assortment', aggfunc=np.sum)
assortment_pivot_total_sales

In [ ]:
assortment_pivot_total_sales.plot(kind='line', title='Total Sales by Month and Store Assortment', figsize=(7,8), grid=True)
plt.show()

A quick look at Sales by volume of total sales shows that stores of Assortment types A and C have significantly more volume than type B stores. Type B stores stay fairly consistent in total Sales volume across all months, with minor upticks during mid-year and end year. Type A and C stores can be seen to follow very similar trends in terms of Sales volume.

A quick look at how many stores we have of each Assortment type will show us significantly less stores of Assortment type B, which accounts for the significantly lower volume of Sales.

In [ ]:
pd.pivot_table((sales_treated[sales_treated['Year'] < 2015]), index='Assortment', values='Store', aggfunc='count')

Due to the vast differences in total Sales volume based on Assortment type, lets also look at the average number of Sales.

(Note: We could include our 2015 data since we're calculating the mean Sales now, but for the sake of consistency when comparing it with the total Sales we will continue to use the same 2013-2014 data.)

In [ ]:
assortment_pivot_avg_sales = pd.pivot_table((sales_treated[sales_treated['Year'] < 2015]), index='Month', values='Sales', columns='Assortment', aggfunc=np.mean)
assortment_pivot_avg_sales

In [ ]:
assortment_pivot_avg_sales.plot(kind='line', title='Average Sales by Month and Store Assortment', figsize=(7,5), grid=True)
plt.show()

Look at average Sales by store Assortment type we can see that stores of type B actually perform quite well when compared to types A and C, despite there being significantly less type B stores! Types A and C continue to follow very similar trends for Sales, but Type C stores consistently outperform type A stores.



## **Question 7: How UPT metric compares across stores of different assortment types ?**

Let's also take a look at how our UPT metric compares across stores of different Assortment types.

(Note: Because we included a UPT metric for our table that still has our extremely high Sales outliers, we will plot that too, to compare.)

In [ ]:
assortment_pivot_avg_UPT = pd.pivot_table((sales_treated[sales_treated['Year'] < 2015]), 
                                     index='Month', values='UPT', columns='Assortment', aggfunc=np.mean)
assortment_pivot_avg_UPT_outliers = pd.pivot_table((sales[sales['Year'] < 2015]), 
                                     index='Month', values='UPT', columns='Assortment', aggfunc=np.mean)

fig, axes = plt.subplots(nrows=1, ncols=2)

assortment_pivot_avg_UPT.plot(kind='line', title='Average UPT by Month and Store Assortment (no outliers)', figsize=(7,5), grid=True, ax=axes[0])
assortment_pivot_avg_UPT_outliers.plot(kind='line', title='Average UPT by Month and Store Assortment (outliers incl.)', figsize=(7,5), grid=True, ax=axes[1])
fig.set_size_inches(15, 5)
plt.show()

We can see stores of Assortment type C are our best performers for UPT. Comparing UPT with and without outliers treated, we can see that with the outliers treated we can more clearly see upward and downward trends, whereas with outliers included these trends look less impactful.

Exploring out Sales and Customers outliers prompted us to create out UPT metric when comparing them by StoreType, so let's explore Sales and UPT by month and StoreType as well.

Let's start with a look at how many stores of each StoreType we have.

In [ ]:
pd.pivot_table((sales_treated[sales_treated['Year'] < 2015]), index='StoreType', values='Store', aggfunc='count')

Next let's look at average Sales per month, broken down by StoreType.

In [ ]:
stype_pivot_avg_sales = pd.pivot_table((sales_treated[sales_treated['Year'] < 2015]), ## Excluding 2015 data
               index='Month', values='Sales', columns='StoreType', aggfunc=np.mean)
stype_pivot_avg_sales

In [ ]:
stype_pivot_avg_sales.plot(kind='line', title='Average Sales by Month and Store Type', figsize=(7,5), grid=True)
plt.show()

We can see that stores of type A, C, and D closely follow very similar trends, whereas stores of type B significantly outperform them when it comes to the average number of Sales.

Now let's see how UPT compares across Store Types.

In [ ]:
stype_pivot_avg_UPT = pd.pivot_table((sales_treated[sales_treated['Year'] < 2015]), 
                                     index='Month', values='UPT', columns='StoreType', aggfunc=np.mean)
stype_pivot_avg_UPT_outliers = pd.pivot_table((sales[sales['Year'] < 2015]), 
                                     index='Month', values='UPT', columns='StoreType', aggfunc=np.mean)

fig, axes = plt.subplots(nrows=1, ncols=2)

stype_pivot_avg_UPT.plot(kind='line', title='Average UPT by Month and Store Type (no outliers)', figsize=(7,5), grid=True, ax=axes[0])
stype_pivot_avg_UPT_outliers.plot(kind='line', title='Average UPT by Month and Store Type (outliers incl.)', figsize=(7,5), grid=True, ax=axes[1])
fig.set_size_inches(15, 5)
plt.show()


Similar to our look at UPT across stores of different Assortment types, we can see that treating the outliers leaves us with a graph that more clearly shows changes in the trend of monthly sales. As we look at average UPT by StoreType, we see that while they follow similar trends, store of StoreType D see customers purchasing approximately 3 more items per purchase on average than at a type A or C store, and approximately 6 more items per purchase on average than customers at type B stores.

It seems customers of type B stores buy less items per purchase on average, but overall type B stores see more sales. It would reason that type B stores must see more customers on average to account for high average sales.

Below we look at total customers and average customers by month. We know of the different store types that there are the fewest stores of type B, so we aren't surprised to see them at the bottom of the Total Customers chart. However, if we look at average customers by store type we see that they average far more customers than the other store types.

In [ ]:
stype_pivot_total_cust = pd.pivot_table((sales_treated[sales_treated['Year'] < 2015]), index='Month', columns='StoreType', values='Customers', aggfunc='count')
stype_pivot_avg_cust = pd.pivot_table((sales_treated[sales_treated['Year'] < 2015]), index='Month', columns='StoreType', values='Customers', aggfunc=np.mean)

fig, axes = plt.subplots(nrows=1, ncols=2)

stype_pivot_total_cust.plot(kind='line', title='Total Customers by Month and Store Type', figsize=(7,5), grid=True, ax=axes[0])
stype_pivot_avg_cust.plot(kind='line', title='Average Customers by Month and Store Type', figsize=(7,5), grid=True, ax=axes[1])
fig.set_size_inches(15, 5)
plt.show()

To understand a bit more about our different Store Types, let's also quickly see how they compare in relation to the Competition Distance.


In [ ]:
stype_pivot_avg_compdist = pd.pivot_table((sales_treated[sales_treated['Year'] < 2015]),
                                         index='StoreType', values='CompetitionDistance', aggfunc=np.mean)

stype_pivot_avg_compdist.plot(kind='bar', rot=0)
plt.show()

Stores of Store Type B are significantly closer to their nearest competitor store, on average. This might suggest the Type B stores are most often in dense urban shopping areas. Perhaps with many other stores available in closer proximity, customers are less likely but multiple items at the store when they can more easily purchase additional items at other nearby stores. Let's see if there's any correlation between CompetitionDistance and UPT.

## **Question 8 : Correlation between competition distance and UPT metric?**

In [ ]:
sales_treated.plot(x='CompetitionDistance', y='UPT', kind='scatter', figsize=(8,6))
plt.show()

We don't see a strong positive or negative correlation between CompetitionDistance and UPT on our scatter plot, but we do see what look to be outliers in our UPT metric.

In [ ]:
corr = sales_treated.corr()

plt.figure(figsize = (15,8))
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, center=0, cmap="YlGnBu", annot=True)
plt.show()

Looking at the correlation calculations above we don't see any meaningful correlations with CompetitionDistance. The notable correlations are a strong positive correlation between Customers and Sales, which isn't surprising. In addition, we see a smaller but positive correlation between Promo and Sales.

# **EDA Conclusion**

**1)**When looking at key performance indicators across store Assortments and Store Types we see they follow similar monthly trends, but numbers can vary by Assortment and Store Type.

**2)**When looking at store Assortment we found that stores of Assortment Type B represent a small share of total sales, which isn’t surprising considering less than 1% of stores are of the Type B assortment. However, when looking at average monthly sales we see the Assortment Type B stores out performing types A and C by a large margin. Looking at the number of units sold per transaction, UPT, we see Assortment Type A and C stores performing better than Type B.

**3)**A similar investigation into sales by Store Type also saw noticeable differences between Store Type B and Store Types A, C, and D. Again, Type B is the least common Store Type, representing only 1.8% of stores sampled from 2013-2014. And yet, similar to Assortment Type B, Store Type B also vastly outperforms other Store Types when looking at average monthly sales. Following the similarities, it also lags to the bottom when we compare UPT amongst Store Types.

**4)**In trying to further investigate that difference between average sales and average UPT for Store Type B, we looked at average customers by Store Type and found Type B to be well ahead of the others. This suggests that Type B stores on average have more customers visit, but they buy a smaller number of items. In looking at the average distance of the nearest competitor by store type we found that Type B stores were far closer to their nearest competitor store on average. This lead us to hypothesize that Type B stores may be concentrated in dense urban areas that see more foot traffic.

**5)**Further along that line of investigation, we looked for any correlations with competition distance, but found no strong correlations. Looking at correlations across our complete data set we only found the expected strong correlation between Customers and Sales, and smaller correlations between Promo and Sales.

**6)**We conclude that well Store Type B and store Assortment B represent a very small sample of the stores, they significantly outperform other store types and assortments in average sales, despite a lower UPT, and see a high volume of customers. It could be worthwhile investigating expanding into more Type B Stores and Assortments.



In [ ]:
sales_treated.info()

In [ ]:
sales_treated = sales_treated.astype({"Promo2SinceWeek": int,'Promo2SinceYear':int,'CompetitionOpenSinceMonth':int,'CompetitionOpenSinceYear':int})

In [ ]:
sales_treated.info()

Lets take out the categorical variable names


In [ ]:
cat_cols = sales_treated.select_dtypes(include=['object']).columns
cat_cols

In [ ]:
# lets check the value counts of each columns
for col in cat_cols:
  print(sales_treated[col].value_counts().sort_index())

Lest change the values of Promointerval column like Feb,May,Aug,Nov - Feb-Nov

In [ ]:
unique_val = sales_treated['PromoInterval'].unique()
unique_val

In [ ]:
for val in unique_val[1:]:
  name=val.split(',')
  new_name = name[0]+'-'+name[-1]
  sales_treated.loc[sales_treated['PromoInterval']==val,'PromoInterval']=new_name


In [ ]:
sales_treated['PromoInterval'].unique()

In [ ]:
# lets one hotencode the categorical variable and save it as different database "reg_dataset"
reg_dataset = pd.get_dummies(sales_treated, columns=cat_cols, prefix=[pref for pref in cat_cols])

In [ ]:
sales_treated.columns

In [ ]:
sales_treated.info()

As we have the Day of month , month and year as the separate column lets first drop the Date column

In [ ]:
reg_dataset.drop('Date',inplace=True,axis = 1)

In [ ]:
reg_dataset.info()

### **Lets apply the linear regression model taking whole data into consideration without identifying feature correlation**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

One Fold regression model

In [ ]:
def simple_regression(data):
  # Get the dependent and independent variable
  dependent_col = 'Sales'
  independent_cols = set(data.columns) - {dependent_col}
  X = data.loc[:,independent_cols].values
  y = data.loc[:,dependent_col].values.reshape(-1,1)
  # lets split the data into training set and test set
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)
  # lets feature scale the data
  scaler = MinMaxScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  # lets fit the data into Linear regression model
  regression = LinearRegression()
  regression.fit(X_train,y_train)
  y_train_pred = regression.predict(X_train)
  y_test_pred = regression.predict(X_test)
  # lets check the mean squared error
  mse_train = mean_squared_error(y_train,y_train_pred)
  mse_test = mean_squared_error(y_test,y_test_pred)
  print(f'MSE train :{mse_train}')
  print(f'MSE test : {mse_test}')
  # lets see the root mean squared error
  rmse_train = np.sqrt(mse_train)
  rmse_test = np.sqrt(mse_test)
  print(f'RMSE train :{rmse_train}')
  print(f'RMSE test : {rmse_test}')
  r2 = r2_score(y_train,y_train_pred)
  print("R2_train :" ,r2)
  print("Adjusted R2_train : ",1-(1-r2_score((y_train),(y_train_pred)))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1)))
  r2 = r2_score(y_test,y_test_pred)
  print("R2_test :" ,r2)
  print("Adjusted R2_test : ",1-(1-r2_score((y_test),(y_test_pred)))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))
  return X,y


In [ ]:
# Get the dependent and independent variable
dependent_col = 'Sales'
independent_cols = set(reg_dataset.columns) - {dependent_col}
X = reg_dataset.loc[:,independent_cols].values
y = reg_dataset.loc[:,dependent_col].values.reshape(-1,1)

In [ ]:
# lets split the data into training set and test set
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
# lets feature scale the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# lets fit the data into Linear regression model
regression = LinearRegression()
regression.fit(X_train,y_train)
y_train_pred = regression.predict(X_train)
y_test_pred = regression.predict(X_test)

In [ ]:
# lets check the mean squared error
mse_train = mean_squared_error(y_train,y_train_pred)
mse_test = mean_squared_error(y_test,y_test_pred)
print(f'MSE train :{mse_train}')
print(f'MSE test : {mse_test}')

In [ ]:
# lets see the root mean squared error
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)
print(f'RMSE train :{rmse_train}')
print(f'RMSE test : {rmse_test}')

As we got the, RMS value lets compare these error with the mean value of Sales column

In [ ]:
y_test.mean()

Lest see what is the proportion of the error wrt mean value

In [ ]:
(rmse_test/y_test.mean())*100

we can see the predicted mean data error is 7.26% of the mean value. Lets check the r2 score

In [ ]:
r2 = r2_score(y_train,y_train_pred)
print("R2_train :" ,r2)
print("Adjusted R2_train : ",1-(1-r2_score((y_train),(y_train_pred)))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1)))
r2 = r2_score(y_test,y_test_pred)
print("R2_test :" ,r2)
print("Adjusted R2_test : ",1-(1-r2_score((y_test),(y_test_pred)))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

# **We can see that the r2_score for both training set and test set is nearly similar and r2_score value is also close to 1...From this we can conclude that our model not only peroformed well on training set but also performed well on test dataset.. And it shows that the model is not underfitting as well as not overfitting.**

**While splitting the data we can get such a split which may give us the good result by so to avoid such by chance result lets split up tha data in 5 part and then apply the regression model and then avarge the mean_squred error and check**

In [ ]:
X_split = np.array_split(X,5)
y_split = np.array_split(y,5)

In [ ]:
def metrics_score(test_data,pred_data,mse_values,rmse_values,r2_score_values,mae_values):
  mse = mean_squared_error(test_data,pred_data)
  rmse = math.sqrt(mean_squared_error(test_data,pred_data))
  r2_score_ = r2_score(test_data,pred_data)
  mae = np.mean(np.abs(test_data-pred_data))
  print(f'MSE is {mse}')
  print(f"RMSE is {rmse}")
  print(f"r2score is {r2_score_}")
  print(f'MAE is {mae}')
  mse_values.append(mse)
  rmse_values.append(rmse)
  r2_score_values.append(r2_score_)
  mae_values.append(mae)
  

Lets make the regression function

In [ ]:
def regression_func(X,y):
  mse_values = [] # mean squared error
  rmse_values=[] # root mean squared error
  r2_score_values = [] # r2_score
  mae_values = [] # mean absolute error
  for i in range(len(X)):
    X_test = X[i]
    y_test = y[i]
    X_train = []
    y_train = []
    for j,data in enumerate(X):
      if i!=j:
        X_train.append(data)
        y_train.append(y[j])
    X_train = np.concatenate(X_train,axis=0)
    y_train = np.concatenate(y_train,axis=0)
    regressor = LinearRegression()
    regressor.fit(X_train,y_train)
    y_pred = regressor.predict(X_test)
    print(f'For {i+1}st Fold')
    metrics_score(y_test,y_pred,mse_values,rmse_values,r2_score_values,mae_values)
  print(f'Final Result:')
  print(f'Mean Squared Error; {np.mean(mse_values)}')
  print(f'Root Mean Squared Error: {np.mean(rmse_values)}')
  print(f'R2 Score Values: {np.mean(r2_score_values)}')
  print(f'Mean Absolute Error: {np.mean(mae_values)}')
 






As I have already splitted X and y in 5 different dataset. Lets run cross fold validation on each fold

In [ ]:
regression_func(X_split,y_split)

**From above information we can see the on every fold we got the approx 0.96 and the Avg. r2 score is 0.96 which is represent good model for prediction**

## **Now lets have a look at the correlated features**

In [ ]:
exclude_col = 'Sales'
include_col = set(sales_treated.columns)-{exclude_col}
corr = sales_treated[include_col].corr()

plt.figure(figsize = (15,8))
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, center=0, cmap="YlGnBu", annot=True)
plt.show()

We can observe that columns **Promo2sinceWeek** & **Promo2SinceYear**,**Promo2sinceYear** and **Promo2**,**Promo2** and **Promo2sinceweek** showing some kind of correlation... Lets plot and see them visually

In [ ]:
correlated_columns = ['Promo2SinceWeek','Promo2SinceYear','Promo2']
fig,axes = plt.subplots(nrows=2,ncols=2,figsize=(15,15))
for i,col in enumerate(correlated_columns[:-1]):
  for j in range(i+1,len(correlated_columns)):
    feature = sales_treated[col]
    label = sales_treated[correlated_columns[j]]
    correlation = feature.corr(label)
    axes[i][j-1].scatter(x=feature, y=label)
    axes[i][j-1].set(xlabel=col, ylabel=correlated_columns[j])
    axes[i][j-1].set_title(correlated_columns[j]+ ' vs ' + col + '- correlation: ' + str(correlation))
    z = np.polyfit(sales_treated[col], sales_treated[correlated_columns[j]], 1)
    y_hat = np.poly1d(z)(sales_treated[col])

    axes[i][j-1].plot(sales_treated[col], y_hat, "r--", lw=1)

plt.show()

Lets check the variance inflation factor for these 3 columns.. ["Promo2","Promo2SinceYear","Promo2SinceWeek"]

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

In [ ]:
print(calc_vif(sales_treated[['Promo2','Promo2SinceYear','Promo2SinceWeek']]))
print(sales_treated['Promo2'].corr(sales_treated['Promo2SinceWeek']))
print(sales_treated['Promo2'].corr(sales_treated['Promo2SinceYear']))

from the above results I can see that Promo2 has the highest variance inflation factor...And even promo2 has significant positive corrlation with the Promo2Since week and very high negative correlation so.... I have decided to drop the promo2 column from regression_data and make a new copy of it

In [ ]:
reg_data_clean = reg_dataset.copy()
reg_data_clean.drop(columns=['Promo2'],inplace=True)


In [ ]:
reg_data_clean.info()

# <p>Lets apply the simple linear regression model on this cleaned dataset </p>

In [ ]:
X,y = simple_regression(reg_data_clean)

## From above results we can see that there is not much significant difference on the result compared to the previous results if we go by performance ..But if feature selection is our main aim in that case we should go with this model...

 <p>Lets run 5 fold cross validation</p>

In [ ]:
X_split = np.array_split(X,5)
y_split = np.array_split(y,5)

In [ ]:
regression_func(X_split,y_split)

# Lets apply the regularization model...

## **Lasso Regression**
We know that Lasso regularization is used to penalize the regression coefficints in such a way that the model should not be overfitted and to avoid multicollinerity by doing the feature selection by making the regression coefficeint of unnecessary feature close to zero sometimes even = 0. In lasso regularization, model tries to minimize the loss fucntion using aboslute sum of regression weights

Lets apply the hyperparameter tuning and try to find out the suitable alpha using **GridSearchCV**<br>


In [ ]:
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import GridSearchCV,HalvingGridSearchCV
from sklearn.linear_model import Lasso

In [ ]:
dependent_col = 'Sales'
independent_cols = set(reg_dataset.columns) - {dependent_col}
X = reg_dataset.loc[:,independent_cols].values
y = reg_dataset.loc[:,dependent_col].values.reshape(-1,1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
### Cross validation
lasso = Lasso()
parameters = {'alpha': [0.0039,0.004,0.0041]}
lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv=5)
lasso_regressor.fit(X_train, y_train)

In [ ]:
print("The best fit alpha value is found out to be :" ,lasso_regressor.best_params_)
print("\nUsing ",lasso_regressor.best_params_, " the negative mean squared error is: ", lasso_regressor.best_score_)

In [ ]:
# lets see no of features included in best model
lasso_regressor.n_features_in_

In [ ]:

# So lets apply the lasso regularization on reg_dataset without dropping promo2 column
# Get the dependent and independent variable
def lasso_regression(data,alpha):
  # Get the dependent and independent variable
  dependent_col = 'Sales'
  independent_cols = list(set(data.columns) - {dependent_col})
  X = data.loc[:,independent_cols].values
  y = data.loc[:,dependent_col].values.reshape(-1,1)
  # lets split the data into training set and test set
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)
  # lets feature scale the data
  scaler = MinMaxScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  # lets fit the data into Linear regression model
  lasso  = Lasso(alpha=alpha , max_iter= 3000)
  lasso.fit(X_train,y_train)
  y_train_pred = lasso.predict(X_train)
  y_test_pred = lasso.predict(X_test)
  # lets get the lasso score
  lasso_train = lasso.score(X_train,y_train)
  print(f'Lasso Score for Training Set : {lasso_train}')
  # lets check the mean squared error
  mse_train = mean_squared_error(y_train,y_train_pred)
  mse_test = mean_squared_error(y_test,y_test_pred)
  print(f'MSE train :{mse_train}')
  print(f'MSE test : {mse_test}')
  # lets see the root mean squared error
  rmse_train = np.sqrt(mse_train)
  rmse_test = np.sqrt(mse_test)
  print(f'RMSE train :{rmse_train}')
  print(f'RMSE test : {rmse_test}')
  r2_train = r2_score(y_train,y_train_pred)
  print("R2_train :" ,r2)
  print("Adjusted R2_train : ",1-(1-r2_score((y_train),(y_train_pred)))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1)))
  r2_test = r2_score(y_test,y_test_pred)
  print("R2_test :" ,r2)
  print("Adjusted R2_test : ",1-(1-r2_score((y_test),(y_test_pred)))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))
  return lasso_train,mse_train,mse_test,rmse_train,rmse_test,r2_train,r2_test,lasso.coef_,independent_cols

In [ ]:
#lets run the lasso regression with alpha=0.004
lasso_score,mse_train,mse_test,rmse_train,rmse_test,r2_train,r2_test,coef,columns = lasso_regression(reg_dataset,0.004)


In [ ]:
# lets check the lasso regression coef
cols = columns
df = pd.DataFrame(list(zip(cols,coef)),columns =['features','coeficients'])

In [ ]:
df

We can see that the Lasso has unselected some unrelevant features by making the coef_ = 0.. And more than that Lasso performed better than Simple linear regression

# **Lets See With Ridge Regression**

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
### Cross validation
ridge = Ridge()
parameters = {'alpha': [1e-15,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1,5,10,20,30,40,45,50,55,60,100]}
ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv=5)
ridge_regressor.fit(X_train, y_train)

In [ ]:
print("The best fit alpha value is found out to be :" ,ridge_regressor.best_params_)
print("\nUsing ",ridge_regressor.best_params_, " the negative mean squared error is: ", ridge_regressor.best_score_)

In [ ]:
### Cross validation
ridge = Ridge()
parameters = {'alpha': [1,2,3,4,5,6]}
ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv=5)
ridge_regressor.fit(X_train, y_train)

In [ ]:
print("The best fit alpha value is found out to be :" ,ridge_regressor.best_params_)
print("\nUsing ",ridge_regressor.best_params_, " the negative mean squared error is: ", ridge_regressor.best_score_)

# <p> So we got the  best alpha parameter for ridge regression is 4 </p>

Running the ridge regression model with alpha = 4

In [ ]:
def ridge_regression(data,alpha):
  # Get the dependent and independent variable
  dependent_col = 'Sales'
  independent_cols = list(set(data.columns) - {dependent_col})
  X = data.loc[:,independent_cols].values
  y = data.loc[:,dependent_col].values.reshape(-1,1)
  # lets split the data into training set and test set
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)
  # lets feature scale the data
  scaler = MinMaxScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  # lets fit the data into Linear regression model
  ridge  = Ridge(alpha=alpha , max_iter= 3000)
  ridge.fit(X_train,y_train)
  y_train_pred = ridge.predict(X_train)
  y_test_pred = ridge.predict(X_test)
  # lets get the lasso score
  ridge_score = ridge.score(X_train,y_train)
  print(f'Ridge Score for Training Set : {ridge_score}')
  # lets check the mean squared error
  mse_train = mean_squared_error(y_train,y_train_pred)
  mse_test = mean_squared_error(y_test,y_test_pred)
  print(f'MSE train :{mse_train}')
  print(f'MSE test : {mse_test}')
  # lets see the root mean squared error
  rmse_train = np.sqrt(mse_train)
  rmse_test = np.sqrt(mse_test)
  print(f'RMSE train :{rmse_train}')
  print(f'RMSE test : {rmse_test}')
  r2_train = r2_score(y_train,y_train_pred)
  print("R2_train :" ,r2)
  print("Adjusted R2_train : ",1-(1-r2_score((y_train),(y_train_pred)))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1)))
  r2_test = r2_score(y_test,y_test_pred)
  print("R2_test :" ,r2)
  print("Adjusted R2_test : ",1-(1-r2_score((y_test),(y_test_pred)))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))
  return ridge_score,mse_train,mse_test,rmse_train,rmse_test,r2_train,r2_test,ridge.coef_,independent_cols

In [ ]:
# lets run the ridge regression with alpha= 4
ridge_score,mse_train,mse_test,rmse_train,rmse_test,r2_train,r2_test,coef,columns = ridge_regression(reg_dataset,4)


In [ ]:
coef[0]

In [ ]:
cols = columns
df = pd.DataFrame(list(zip(cols,coef[0])),columns =['features','coeficients'])
df